In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as scp

# Getting the protein sequences to run DeepSig with

Downloaded the translated amino-acid sequence fasta from https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M28/gencode.vM28.pc_translations.fa.gz on 3/8/2022

In [114]:
with open('../preprocessing/MOP_preprocessing/mop_genes.txt') as mop_gene_f:
    mop_genes = set([g.strip() for g in mop_gene_f.readlines()])
    
with open('../preprocessing/Vizgen_preprocessing/vizgen_genes.txt') as viz_gene_f:
    viz_genes = set([g.strip() for g in viz_gene_f.readlines()])

with open('../preprocessing/SeqFishplus_preprocessing/seq_fish_plus_genes.txt') as seq_gene_f:
    seq_genes = set([g.strip() for g in seq_gene_f.readlines()])

    
genes = mop_genes.union(viz_genes).union(seq_genes)
print(len(genes))

10375


In [128]:
import pysam

path = '/oak/stanford/groups/horence/rob/isoform_localizations/deepsig/gencode.vM28.pc_translations.fa'

seqs = {}

with pysam.FastaFile(path) as fa:
    for ref in fa.references:
        gene = ref.split('|')[-2]
        if gene not in genes:
            continue
        
        seq = fa.fetch(reference=ref)
        
        #use the longest isoform for the signal peptide prediction        
        if (gene not in seqs) or (len(seq) > len(seqs[gene])):
            seqs[gene] = seq
            
        
len(seqs)

9974

In [129]:
#Nearly all the MOp genes were found
print(len(mop_genes))
len(mop_genes.intersection(seqs.keys()))

252


238

In [130]:
#Most of the Viz genes were found
print(len(viz_genes))
len(viz_genes.intersection(seqs.keys()))

649


478

In [132]:
#Most of the Seq genes were found
print(len(seq_genes))
len(seq_genes.intersection(seqs.keys()))

10000


9772

In [133]:
with open('/oak/stanford/groups/horence/rob/isoform_localizations/deepsig/sub_prots.fa','w') as fa:
    for gene,seq in seqs.items():
        fa.write('>{}\n{}\n'.format(gene,seq))

# Notes on running Deepsig

Working in this directory: /oak/stanford/groups/horence/rob/isoform_localizations/deepsig

Have a virtualenv with deepsig installed: source .venv/bin/activate

(need to ml python/3.9.0 too)

then deepsig -h to get the help menu

I ran deepsig with: deepsig -f sub_prots.fa -k euk -o results.txt

Then copied the outputs to SRRS/inputs/deepsig_results.csv